In [4]:
import pyautogui
from PIL import Image
import pydirectinput
import time

def change():
    # Press the 'l' key
    pydirectinput.press('4')
    #print('l key pressed')

    # Wait for 0.1 seconds
    time.sleep(0.1)

screen_width, screen_height = pyautogui.size()

screenshot_width = 1780
screenshot_height = 1328

# Calculate the top-left corner of the screenshot region
x = (screen_width - screenshot_width) // 2
y = (screen_height - screenshot_height) // 2

def printScreen(counter):
    # Take a screenshot of the region
    screenshot = pyautogui.screenshot(region=(x, y, screenshot_width, screenshot_height))

    # Save the screenshot
    screenshot.save("C:\\Users\\Eduardo\\Pictures\\Screenshots\\persona3_personas\{0}.png".format(counter))


def begin():
    counter = 1
    time.sleep(5)
    while counter < 200:
        change()
        printScreen(counter)
        counter += 1

time.sleep(5)
begin()

KeyboardInterrupt: 

In [2]:
import pyautogui
import time
# Get screen size
screen_width, screen_height = pyautogui.size()

# Define the size of the screenshot
screenshot_width = 1780
screenshot_height = 1328

# Calculate the top-left corner of the screenshot region
x = (screen_width - screenshot_width) // 2
y = (screen_height - screenshot_height) // 2

# Take a screenshot of the region
screenshot = pyautogui.screenshot(region=(x, y, screenshot_width, screenshot_height))

# Save the screenshot
screenshot.save("C:\\Users\\Eduardo\\Pictures\\Screenshots\\persona3_personas\\screenshot.png")

In [52]:
import pytesseract
from PIL import Image
import re
import numpy as np
import re

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # replace with your actual path
# Custom configuration for pytesseract

# Function to process an image and extract persona information into a dictionary


class Rectangle:
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
    def __str__(self):
        return f"Rectangle(x={self.x}, y={self.y}, width={self.width}, height={self.height})"


rectangle_list = []

rectange_up_left = (99,933)

rectange_down_right = (610,1000)

x = rectange_up_left[0]
y = rectange_up_left[1]
width = rectange_down_right[0] - x
height = rectange_down_right[1] - y

#skill rectangles
for i in range(4):
    rec = Rectangle(x,y + (76 * i), width, height)
    rectangle_list.append(rec)

x = 628
for i in range(4):
    rec = Rectangle(x,y + (76 * i), width, height)
    rectangle_list.append(rec)

# level rectangle
level_rec_up_left = (223,649)
level_rec_down_right = (309,696)

x = level_rec_up_left[0]
y = level_rec_up_left[1]
width = level_rec_down_right[0] - x
height = level_rec_down_right[1] - y
rec = Rectangle(x,y, width, height)
rectangle_list.append(rec)

# name rectangle
name_rec_up_left = (316,626)
name_rec_down_right = (837,690)
x = name_rec_up_left[0]
y = name_rec_up_left[1]
width = name_rec_down_right[0] - x
height = name_rec_down_right[1] - y
rec = Rectangle(x,y, width, height)
rectangle_list.append(rec)


class Persona:

    def __init__(self, name, level, skills, counter):
        self.name = name
        self.level = level
        self.skills = skills
        self.counter = counter

    def __str__(self):
        return f"Persona(name={self.name}, level={self.level}, skills={self.skills})"

    def __repr__(self):
        return f"Persona(name={self.name}, level={self.level}, skills={self.skills})"

personas = []

found_skills = {}

for counter in range(1,143):
    image = Image.open("C:\\Users\\Eduardo\\Pictures\\Screenshots\\persona3_personas\{0}.png".format(counter))
    # using the rectangle list, crop the image in each rectangle
    rec_counter = 0

    skills = []
    name = None
    level = None

    for rectangle in rectangle_list:
        cropped_image = image.crop((rectangle.x,rectangle.y,rectangle.x + rectangle.width,rectangle.y + rectangle.height))

        rgb_img = cropped_image.convert('RGB')

        if rec_counter < 8:
            hex_color = '7f9fff'
        elif rec_counter == 8:
            hex_color = '083959'
        elif rec_counter == 9:
            hex_color = '18315e'

        # Convert the target hex color to RGB
        target_r, target_g, target_b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        target_color = np.array([target_r, target_g, target_b])

        # Convert the image to a NumPy array
        img_array = np.array(rgb_img)

        # Calculate the Euclidean distance between each pixel's color and the target color
        color_distance = np.linalg.norm(img_array - target_color, axis=2)

        # Create a mask for pixels that are similar to the target color
        mask = color_distance < 65

        # Change the color of the pixels based on the mask
        img_array[mask] = [0, 0, 0]  # black
        img_array[~mask] = [255, 255, 255]  # white

        # Convert the NumPy array back to an image
        rgb_img = Image.fromarray(img_array)

        # Convert the image to grayscale
        gray_img = rgb_img.convert('L')

        # Apply a threshold to make it binary
        binary_img = gray_img.point(lambda x: 0 if x<128 else 255, '1')
        
        if rec_counter < 8:
            skill = pytesseract.image_to_string(binary_img, config='--psm 13').strip('\n')

            # 'remove all characters that are not letters, numbers, ' ' or '-'
            skill = re.sub(r'[^a-zA-Z0-9 -]', '', skill)
            

            # remove ' ' and '-' from the beginning and end of the string
            skill = skill.strip(' -')
            
            if found_skills.get(skill) is None:
                # save the image 'cropped_image' to a file with the filename being the value in the 'skill' variable
                cropped_image.save("C:\\Users\\Eduardo\\Pictures\\Screenshots\\persona3_personas\\skills_raw\\{0}.png".format(skill))
                found_skills[skill] = True

            
            print(skill)
            skills.append(skill)
        elif rec_counter == 8:
            # Run OCR on the image
            raw_level = pytesseract.image_to_string(binary_img, config='--psm 13')

            # Use regex to find all numbers in the output
            numbers = re.findall(r'\d+', raw_level)

            # If any numbers were found, choose the first one
            if numbers:
                level = int(numbers[0])

                # Clamp the level to the range 1-99
                level = max(1, min(level, 99))
            else:
                # If no numbers were found, default to 1
                level = -1
            print(level)
        elif rec_counter == 9:
            name = pytesseract.image_to_string(binary_img, config='--psm 13')
            print(name)

        rec_counter += 1
    
    persona = Persona(name, level, skills, counter)

    print('----------------------------------------------------------')

    personas.append(persona)

Bash
Agi
a
a
Be
BS
a
a
-1
Orpheus sits

----------------------------------------------------------
Bufu sits
Posumudi
Sukunda
a
Be
BS
a
a
5
-ApsaraS sits”

----------------------------------------------------------
Agi
Cleave
a
a
Be
BS
a
a
5
Nekomata —s—©»

----------------------------------------------------------
Dia
Sukunda
Garu
a
Be
BS
a
a
-1
Alp ———”

----------------------------------------------------------
Zio
Dia ss
a
a
Be
BS
a
a
-1
Omoikane ss

----------------------------------------------------------
Bufu sits
Bash
Tarukaja
a
Be
BS
a
a
-1
Forneus

----------------------------------------------------------
Garuuttit
Patra
Sukukaja
Hama
Charmdi iw
BS
a
a
-1
Angel is

----------------------------------------------------------
Bash
Agi
Rakunda
Sukukaja
Diarama sis
BS
a
a
-1
Ara Mitama >

----------------------------------------------------------
Cleave
Agi
Bewilder
a
Be
BS
a
a
9
Chimera its

----------------------------------------------------------
Poisma
Single Shot
Zio
a
Be


In [53]:
list_skill_raw = []
for persona in personas:
    for skill in persona.skills:
        list_skill_raw.append(skill)

list_skill_raw = list(set(list_skill_raw))

print(len(list_skill_raw))

266


In [54]:
# open 'skills.csv'
import pandas as pd


file_path = "C:\\Users\\Eduardo\\Documents\\Projects\\AutoTelos\\new_telos\\skills.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

print(df.head())

# Create a list of all the skills in the CSV file
list_skill_csv = df['skill'].tolist()

print(list_skill_csv)

         skill                                             learns
0       Cleave     Hermes, Nekomata, Chimera, Archangel, Valkyrie
1  Power Slash                           Valkyrie, Gurr, Rakshasa
2     Getsu-ei  Rakshasa, Ares, Take-Mikazuchi, Orthrus, Hanum...
3       Zan-ei  Oberon, Ares, Titan, Pale Rider, Power, Jikoku...
4   Gale Slash             Rakshasa, Ghoul, Shiisaa, Flauros, Ose
['Cleave', 'Power Slash', 'Getsu-ei', 'Zan-ei', 'Gale Slash', 'Mighty Swing', 'Fatal End', 'Blade of Fury', 'Deathbound', "Heaven's Blade", 'Tempest Slash', 'Brave Blade', 'Scarlet Havoc', 'Vorpal Blade', 'Bash', 'Assault Dive', 'Sonic Punch', 'Swift Strike', 'Kill Rush', 'Herculean Strike', 'Gigantic Fist', 'Heat Wave', 'Vicious Strike', 'Akasha Arts', "God's Hand", 'Shadow Hound', 'Weary Thrust', 'Single Shot', 'Double Fangs', 'Holy Arrow', 'Twin Shot', 'Cruel Attack', 'Torrent Shot', 'Vile Assault', 'Poison Arrow', 'Arrow Rain', 'Myriad Arrows', 'Pralaya', 'Primal Force', 'Agi', 'Maragi', 'Agil

In [55]:
list_skill_raw.sort()
print('{')
for skill in list_skill_raw:
    if skill not in list_skill_csv:
        print(' \'{}\' : \'\','.format(skill) )
print('}')

{
 'Agidyne ss' : '',
 'Amrita t' : '',
 'Auto-Rakuka ja' : '',
 'Auto-Sukuka ja' : '',
 'Auto-Taruka ja' : '',
 'BS' : '',
 'Basho' : '',
 'Be' : '',
 'Bufu sits' : '',
 'Bufu ss' : '',
 'Bufula oo' : '',
 'Bufuo it' : '',
 'Charmdi iw' : '',
 'Dia ss' : '',
 'Diarama sis' : '',
 'Die for Me' : '',
 'Dodge lee' : '',
 'Evade lee' : '',
 'FatalEnd' : '',
 'Garula it' : '',
 'Garuuttit' : '',
 'Gods Hand' : '',
 'Growth 30' : '',
 'Growth To' : '',
 'Growth3' : '',
 'Hama sits' : '',
 'Hama sitse' : '',
 'Hamaon i' : '',
 'Hamas' : '',
 'Heavens Blade' : '',
 'Invigorate 30' : '',
 'JTarukaja' : '',
 'KillRush' : '',
 'Mabufuo' : '',
 'Mabufuoit' : '',
 'Mabufuu' : '',
 'Mahama s' : '',
 'Maravi' : '',
 'Maravi tt' : '',
 'Maravion' : '',
 'Maravion  s' : '',
 'Matarunda s' : '',
 'Mavaruo' : '',
 'Mavgarudyne' : '',
 'Mavgarula os' : '',
 'Mazioo' : '',
 'Media sit' : '',
 'Mictory Cry' : '',
 'Mudo 7' : '',
 'Mudo ss' : '',
 'Null bight' : '',
 'NullElec' : '',
 'NullFear' : '',
 'Nul

In [61]:
update_skills = {
 'Agidyne ss' : 'Agidyne',
 'Amrita t' : 'Amrita',
 'Auto-Rakuka ja' : 'Auto-Rakukaja',
 'Auto-Sukuka ja' : 'Auto-Sukukaja',
 'Auto-Taruka ja' : 'Auto-Tarukaja',
 'BS' : '',
 'Basho' : 'Bash',
 'Be' : '',
 'Bufu sits' : 'Bufu',
 'Bufu ss' : 'Bufu',
 'Bufula oo' : 'Bufula',
 'Bufuo it' : 'Bufu',
 'Charmdi iw' : 'Charmdi',
 'Dia ss' : 'Dia',
 'Diarama sis' : 'Diarama',
 'Die for Me' : 'Die for Me!',
 'Dodge lee' : 'Dodge Ice',
 'Evade lee' : 'Evade Ice',
 'FatalEnd' : 'Fatal End',
 'Garula it' : 'Garula',
 'Garuuttit' : 'Garu',
 'Gods Hand' : 'God\'s Hand',
 'Growth 30' : 'Growth 3',
 'Growth To' : 'Growth 1',
 'Growth3' : 'Growth 3',
 'Hama sits' : 'Hama',
 'Hama sitse' : 'Hama',
 'Hamaon i' : 'Hamaon',
 'Hamas' : 'Hama',
 'Heavens Blade' : 'Heaven\'s Blade',
 'Invigorate 30' : 'Invigorate 3',
 'JTarukaja' : 'Tarukaja',
 'KillRush' : 'Kill Rush',
 'Mabufuo' : 'Mabufu',
 'Mabufuoit' : 'Mabufu',
 'Mabufuu' : 'Mabufu',
 'Mahama s' : 'Mahama',
 'Maravi' : 'Maragi',
 'Maravi tt' : 'Maragi',
 'Maravion' : 'Maragion',
 'Maravion  s' : 'Maragion',
 'Matarunda s' : 'Matarunda',
 'Mavaruo' : 'Magaru',
 'Mavgarudyne' : 'Magarudyne',
 'Mavgarula os' : 'Magarula',
 'Mazioo' : 'Mazio',
 'Media sit' : 'Media',
 'Mictory Cry' : 'Victory Cry',
 'Mudo 7' : 'Mudo',
 'Mudo ss' : 'Mudo',
 'Null bight' : 'Null Light',
 'NullElec' : 'Null Elec',
 'NullFear' : 'Null Fear',
 'Nulllce' : 'Null Ice',
 'Panta Rhet' : 'Panta Rhei',
 'Patraw' : 'Patra',
 'Potson Mist' : 'Poison Mist',
 'Power Charze' : 'Power Charge',
 'Rakunda ite' : 'Rakunda',
 'RePatra st' : 'Re Patra',
 'Recarm 2' : 'Recarm',
 'Sukunda sit' : 'Sukunda',
 'Zan-ci' : 'Zan-ei',
 'Zionga it' : 'Zionga',
 'Zionga st' : 'Zionga',
 'a' : '',
 'thSwd Master' : '1hSwd Master',
}

In [63]:
# iterate by the values of the dictionary and see if any of them is not in the dataframe
for key, value in update_skills.items():
    if value not in list_skill_csv:
        print(key, value)

BS 
Be 
a 


In [64]:
# problematic way, may criple some skills. use list comprehesion instead
for persona in personas:
    for skill in persona.skills:
        if skill in update_skills:
            persona.skills.remove(skill)
            persona.skills.append(update_skills[skill])

In [85]:
list_blanks = [
    'a',
    'BS',
    'Be',
    '',
]

for persona in personas:
    # Create a new list that only includes the skills not in list_blanks
    persona.skills = [skill for skill in persona.skills if skill not in list_blanks]

In [84]:
print(personas[0].skills[2] == 'BS')

True


In [86]:
print('[')
for persona in personas:
    print('{')
    print('    "name": "{0}",'.format(persona.name.strip('\n')))
    print('    "level": {0},'.format(persona.level))
    print('    "skills": [')
    counter = 0
    for skill in persona.skills:
        # if the skill is the last one, don't add a comma
        if counter == len(persona.skills) - 1:
            print('        "{0}"'.format(skill.strip('\n')))
        else:
            print('        "{0}",'.format(skill.strip('\n')))
        counter += 1
    print('    ]')
    print('},')
print(']')

[
{
    "name": "Orpheus sits",
    "level": -1,
    "skills": [
        "Bash",
        "Agi"
    ]
},
{
    "name": "-ApsaraS sits”",
    "level": 5,
    "skills": [
        "Posumudi",
        "Sukunda",
        "Bufu"
    ]
},
{
    "name": "Nekomata —s—©»",
    "level": 5,
    "skills": [
        "Agi",
        "Cleave"
    ]
},
{
    "name": "Alp ———”",
    "level": -1,
    "skills": [
        "Dia",
        "Sukunda",
        "Garu"
    ]
},
{
    "name": "Omoikane ss",
    "level": -1,
    "skills": [
        "Zio",
        "Dia"
    ]
},
{
    "name": "Forneus",
    "level": -1,
    "skills": [
        "Bash",
        "Tarukaja",
        "Bufu"
    ]
},
{
    "name": "Angel is",
    "level": -1,
    "skills": [
        "Patra",
        "Sukukaja",
        "Hama",
        "Garu",
        "Charmdi"
    ]
},
{
    "name": "Ara Mitama >",
    "level": -1,
    "skills": [
        "Bash",
        "Agi",
        "Rakunda",
        "Sukukaja",
        "Diarama"
    ]
},
{
    "name": 

In [114]:
class PersonaJSON:
    def __init__(self, name, level, skills):
        self.name = name
        self.level = level
        self.skills = skills

import json

# Load the JSON file
with open('persona_compendium_save_file.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
# Create Persona objects
personasJSON = [PersonaJSON(item['name'], item['level'], item['skills']) for item in data]

In [115]:
for persona in personasJSON:
    for skill in persona.skills:
        if skill not in list_skill_csv:
            print(skill)
            

In [113]:
all_personas = []

for line in df['learns']:
    list_of_personas = [persona.strip("' ") for persona in line.split(',')]
    for persona in list_of_personas:
        all_personas.append(persona)

all_personas = list(set(all_personas))

all_personas.sort()

print(all_personas)

['Abaddon', 'Alice', 'Alilat', 'Alp', 'Ananta', 'Angel', 'Anubis', 'Apsaras', 'Ara Mitama', 'Arahabaki', 'Archangel', 'Ares', 'Asura', 'Atavaka', 'Atropos', 'Attis', 'Baal Zebul', 'Barong', 'Beelzebub', 'Berith', 'Bishamonten', 'Black Frost', 'Byakko', 'Castor', 'Cerberus', 'Chernobog', 'Chi You', 'Chimera', 'Clotho', 'Cu Chulainn', 'Cybele', 'Daisoujou', 'Decarabia', 'Dionysus', 'Dominion', 'Eligor', 'Empusa', 'Flauros', 'Forneus', 'Fortuna', 'Gabriel', 'Ganesha', 'Ganga', 'Garuda', 'Genbu', 'Ghoul', 'Girimehkala', 'Gurr', 'Hanuman', 'Hariti', 'Hecatoncheires', 'Helel', 'Hell Biker', 'Hermes', 'High Pixie', 'Hokuto Seikun', 'Horus', 'Hua Po', 'Incubus', 'Inugami', 'Io', 'Jack Frost', 'Jatayu', 'Jikokuten', 'Kaiwan', 'Kali', 'Kartikeya', 'Kikuri-Hime', 'King Frost', 'Kingu', 'Kohryu', 'Koumokuten', 'Kumbhanda', 'Kurama Tengu', 'Kusi Mitama', 'Lachesis', 'Laksmi', 'Lamia', 'Leanan Sidhe', 'Legion', 'Lilim', 'Lilith', 'Loa', 'Loki', 'Lucia', 'Lucifer', 'Mara', 'Masakado', 'Melchizedek', 

In [116]:
for persona in personasJSON:
    if persona.name not in all_personas:
        # print persona.name with the correct format
        print('\'{0}\''.format(persona.name))

178
